# Plotting very large datasets meaningfully, using `datashader`

There are a variety of approaches for plotting large datasets, but most of them are very unsatisfactory. Here we first show some of the issues, then demonstrate how the `datashader` library helps make large datasets practical.  

We'll use part of the well-studied [NYC Taxi trip database](http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml), with the locations of all NYC taxi pickups and dropoffs from the month of January 2015.  Although we know what the data is, let's approach it as if we are doing data mining, and see what it takes to understand the dataset from scratch.

## Load NYC Taxi data 

(takes a dozen seconds or so, since it's in a fairly inefficient CSV file format...)

In [ ]:
import pandas as pd

df = pd.read_csv('data/nyc_taxi.csv',usecols=['pickup_x','pickup_y','dropoff_x','dropoff_y','passenger_count'])
x_range = (-8250000,-8210000)
y_range = (4965000,4990000)
df.tail()

As you can see, this file contains about 12 million pickup and dropoff locations (in Web Mercator coordinates), with passenger counts.

## Define a simple plot

In [ ]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.tile_providers import STAMEN_TONER

output_notebook()

def base_plot():
    p = figure(tools='pan,wheel_zoom,box_zoom,reset', 
               plot_width=900, plot_height=600, 
               x_range=x_range, y_range=y_range)
    p.axis.visible = False
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    return p
    
options = dict(line_color='black', fill_color='blue', size=5)

## 1000-point scatterplot: individual points visible

Any plotting program should be able to handle a plot of 1000 datapoints.  Here the points are initially overplotting each other, but if you hit the Reset button (top right of plot) to zoom in a bit, each of them should be clearly visible in the following Bokeh plot of a random 1000-point sample.  If you know what to look for, you can even see the outline of Manhattan Island and Central Park from the pattern of dots.  We've included geographic map data here to help get you situated, though for a genuine data mining task you might not have any such landmarks.  Because this plot is discarding 99.99% of the data, it reveals very little of what might be contained in the dataset.

In [ ]:
samples = df.sample(n=1000)
p = base_plot()
p.add_tile(STAMEN_TONER)
p.circle(x=samples['dropoff_x'], y=samples['dropoff_y'], **options)
show(p)

## 10,000-point scatterplot: overplotting

Even if we try to plot only 0.1% of the data as clearly visible points, there are major problems with overplotting, such that the true density of dropoffs in central Manhattan is impossible to see due to occlusion:

In [ ]:
samples = df.sample(n=10000)
p = base_plot()
p.circle(x=samples['dropoff_x'], y=samples['dropoff_y'], **options)
show(p)

## 100,000-point scatterplot: saturation

If we give up on seeing individual trips, we can make the dot size very small, and we can also set an alpha (transparency) parameter so that overplotted dropoffs will sum up rather than occluding.  Bokeh performs fairly well with 100,000 points (unlike many plotting programs), though plotting is definitely getting slower in this case. (Bokeh generates a very large HTML file to be read by the browser, because it contains each point represented individually).  More importantly, it's very tricky to set the size and alpha parameters.  How do we know if certain regions are saturating so that density differences are obscured?  Here we've set the alpha to show a clear structure of streets and blocks, as one would expect to see, but the density of dropoffs still seems approximately the same on all Manhattan streets (just wider in some locations), which is unlikely to be true.  Plus, this alpha value will work only at the one zoom level shown; if we zoom in there will be less overlap between dropoff locations, and we'll need to lower the transparency of each point.  Worst, we're still ignoring 99% of the data, though the result is starting to look interesting already:

In [ ]:
options = dict(line_color='blue', fill_color='blue', size=1, alpha=0.05)
samples = df.sample(n=100000)
p = base_plot()
p.circle(x=samples['dropoff_x'], y=samples['dropoff_y'], **options)
show(p)

## 10-million-point datashaded plots: auto-ranging, but not perceptually calibrated

Instead of using a Bokeh scatterplot, which encodes every point into JSON and stores it in the HTML file read by the browser, we can use the datashader library to render the entire dataset into a pixel buffer in a separate Python process, always providing a fixed-size image to the browser. The steps involved are to create a Canvas object with the shape of the eventual plot (i.e. having one bin for collecting points, per final pixel), then aggregating all points into that set of bins, incrementally counting them, and then mapping the resulting counts into a visible color from a specified range:

In [ ]:
import datashader as ds
from datashader import transfer_functions as tf
from functools import partial

def create_image(x_range, y_range, w, h, color_fn=tf.interpolate):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    agg = cvs.points(df, 'dropoff_x',  'dropoff_y',  ds.count('passenger_count'))
    image = color_fn(agg)
    return image

Because the number of points involved is no longer a limiting factor, you can use the entire dataset.  Moreover, datashader allows computation on the intermediate stages of plotting, which lets you easily define operations like auto-ranging (which is on by default), so that we can be sure there is no overplotting or saturation and no need to set an alpha parameter. However, when we try it, we can see that the results from this naive approach are unsatisfying:

In [ ]:
create_image(x_range,y_range,800,500, color_fn=partial(tf.interpolate, low="white", high='darkblue', how='linear'))

In the plot above we can clearly see a peak in dropoffs around Times Square, as one might expect, and we know there is no saturation because of datashader's default auto-ranging.  Yet very little of the other structure of the dataset is visible -- most areas *appear* to have no dropoffs. To see if that's really true, let's set the color for pixels that have at least one dropoff as a light color, and then interpolate from there for higher values.  With datashading, it's easy to do that, because one can express computations not just on the data, but on the visualization.  In the resulting plot (below), it is obvious that most areas *do* have dropoffs, and that there is rich structure in this data.  But the result is still quite unsatisfactory -- most regions of Manhattan seem to be nearly uniformly densely populated with dropoffs, apart from the one Times Square peak and some dimly visible street outlines: 

In [ ]:
create_image(x_range,y_range,800,500, color_fn=partial(tf.interpolate, low="lightblue", high='darkblue', how='linear'))

## 10-million-point datashaded plots: perceptually calibrated, and interactive

The key limitation of the above datashaded plots is that they use linear scaling between counts and the colors shown on screen.  Our visual systems do not perceive linear color ramps linearly, and so we cannot see much of the structure present in such plots.  Instead, we need to use nonlinear scaling, such as a log or cube root function mapping from the number of counts into the color ramp.  The datashader default is to use log scaling, which is why the above plots had to specify "linear" explicitly.  Here we'll use the log default, and we'll also embed the generated images into a Bokeh plot to support fully interactive zooming:

In [ ]:
import datashader as ds
from datashader.callbacks import InteractiveImage

p = base_plot()
InteractiveImage(p, create_image)

Finally, the full structure of the data is visible, and it's actually easier to use than all of the previous steps, because no parameters are required.  Just provide the data to plot, and datashader will accurately reveal it to the observer.  Here there are now clearly more dropoffs on streets than on the surrounding regions, and this structure is visible both in central Manhattan (where there are clearly many more dropoffs total) and in the suburbs.  There are also some clear problems with the quality of the data -- there are a significant number of trips that claim to drop off in the water or in the roadless areas of Central park, as well as in the middle of most of the tallest buildings in central Manhattan. These locations are likely to be GPS errors being made visible, perhaps partly because of poor GPS performance in between the tallest buildings.

Note that you can zoom in interactively to this plot, seeing all the points available in that viewport, without ever needing to change the plot parameters.  Each time you zoom or pan, a new image is rendered (which takes a few seconds for large datasets), and displayed overlaid the other plot elements, providing full access to all of your data.

## Customizing datashader

Each of the stages of the datashader pipeline can be changed, either for personal preferences or to highlight specific aspects of the data.  E.g. you can map the counts into a different set of colors.  Here we replace `create_image` with a user-customizable `Pipeline` object that encapsulates this common series of processing steps, making it easy to create an interactive plot from scratch:

In [ ]:
p = base_plot()
pipeline = ds.Pipeline(df=df, glyph=ds.Point("dropoff_x", "dropoff_y"), agg=ds.count("passenger_count"),
                       color_fn=partial(tf.interpolate,low="lightpink",high="darkred"))
InteractiveImage(p, pipeline)

Alternatively, you can arbitrarily specify each of these steps, to do novel operations once your data has been aggregated into pixel-shaped bins -- computations on the visualization, not just the data!  For instance, you might want to plot all the pixels where there were more dropoffs than pickups in blue, and all those where there were more pickups than dropoffs in red.  To do this, just write your own function that will create an image, when given x and y ranges, a resolution (w x h), and zero or more optional arguments.  You can then either call it yourself, or pass it to `InteractiveImage` to make an interactive plot:

In [ ]:
import datashader as ds
from datashader.callbacks import InteractiveImage
from datashader import transfer_functions as tf

def create_image(x_range, y_range, w, h, how='log'):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    picks = cvs.points(df, 'pickup_x',  'pickup_y',  ds.count('passenger_count'))
    drops = cvs.points(df, 'dropoff_x', 'dropoff_y', ds.count('passenger_count'))
    more_drops = tf.interpolate(drops.where(drops > picks), "lightblue", 'blue', how=how)
    more_picks = tf.interpolate(picks.where(picks > drops), "lightpink", 'red',  how=how)
    return tf.stack(more_picks,more_drops)

p = base_plot()
InteractiveImage(p, create_image)

Now you can see that pickups are more common on major roads, as you'd expect, and dropoffs are more common on side streets.  In Manhattan, roads running along the island are more common for pickups. If you zoom in to any location, the data will be re-aggregated to the new resolution automatically, again calculating for each pixel whether pickups or dropoffs were more likely.  

The above example just used pre-existing components provided for the datashader pipeline, but you can implement any components you like and substitute them, allowing you to easily explore and highlight specific aspects of your data. Have fun datashading!